# Combustion Chemistry in Isobaric Reactor: Generatign Training Database in the Principal Component Space

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')


### Importing External Libraries

import numpy                             as np
import pandas                            as pd
pd.options.mode.chained_assignment = None  # default='warn'


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE    
# from PCAfold         import PCA          as PCAA

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir    = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/0DReact_Isobaric_500Cases_H2/')


NVarsRed     = 15

valid_perc   = 20.

FixedMinVal  = 1.e-14

n_ics        = 500


ColorVec     = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

In [3]:
input_vars_        = pd.read_csv(OutputDir+'/'+str(NVarsRed)+'PC/CleanVars_ToRed.csv', header=None).to_numpy()[0,:]
try:
    input_vars_not = pd.read_csv(OutputDir+'/'+str(NVarsRed)+'PC/CleanVars_NotToRed.csv', header=None).to_numpy()[0,:]
except:
    input_vars_not = []
input_vars         = np.append(input_vars_, input_vars_not)

In [4]:
Vars = list(input_vars_not)
for iVars in range(1,NVarsRed+1):
    Vars.append('PC_'+str(iVars))

Vars0    = []
for Var in input_vars_not:
    Vars0.append(Var+'0')
for iVars in range(1,NVarsRed+1):
    Vars0.append('PC0_'+str(iVars))
    
InputVar = ['t'] + Vars0

In [5]:
Header           = Vars[0]
for var in Vars[1:]:
    Header      += ','+var
FileName = OutputDir+'/'+str(NVarsRed)+'PC/Vars.csv'
with open(FileName, 'w') as the_file:
    the_file.write(Header+'\n')

Header           = Vars0[0]
for var in Vars0[1:]:
    Header      += ','+var
FileName = OutputDir+'/'+str(NVarsRed)+'PC/Vars0.csv'
with open(FileName, 'w') as the_file:
    the_file.write(Header+'\n')

In [6]:
# FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
# Data         = pd.read_csv(FileName, header=0)
# SimIdxs      = Data.to_numpy(int)


# FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
# Data         = pd.read_csv(FileName)
# ICVecs       = Data.to_numpy()
# n_ics         = len(ICVecs)


# P0Vec        = []
# EqRatio0Vec  = []
# T0Vec        = []
# for iC in range(n_ics):
#     Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
#     P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
#     EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
#     T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
# P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
# EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
# T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(n_ics):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)


for iC in range(n_ics):
    try:
        FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        DataNew          = pd.read_csv(FileName, header=0)
        Nts              = len(DataNew)
        DataTemp         = DataNew.iloc[[0]][Vars] #DataTemp.iloc[1][Vars]
        DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
        DataTemp.reset_index(drop=True, inplace=True)
        DataNew[Vars0]   = DataTemp
        DataTemp['t']    = DataNew['t']
        DataNew['t']     = np.maximum(DataNew['t'].to_numpy() - DataTemp['t'].iloc[[0]].to_numpy(), FixedMinVal)
        if (iC==0):
            Data         = DataNew
        else:
            Data         = pd.concat([Data, DataNew], axis=0)
    except:
        pass
Data.reset_index(drop=True, inplace=True)

In [7]:
Data

,t,T,H2,O2,N2,PC_1,PC_2,PC_3,PC_4,PC_5,...,PC0_6,PC0_7,PC0_8,PC0_9,PC0_10,PC0_11,PC0_12,PC0_13,PC0_14,PC0_15
0,1.000000e-14,3.292478,-1.134680,-0.686480,-0.142168,-22.862158,14.991065,4.176808,-0.712233,0.830389,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
1,9.999999e-08,3.292474,-1.134682,-0.686491,-0.142168,-3.108088,-1.437114,0.295362,-0.155962,0.415891,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
2,1.032895e-07,3.292474,-1.134682,-0.686492,-0.142168,-3.076747,-1.441802,0.298002,-0.163514,0.411918,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
3,1.066872e-07,3.292474,-1.134682,-0.686492,-0.142168,-3.045443,-1.446481,0.300621,-0.171044,0.407964,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
4,1.101967e-07,3.292474,-1.134682,-0.686492,-0.142168,-3.014175,-1.451149,0.303220,-0.178551,0.404027,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,8.785674e-01,3.424208,-1.277992,-4.147286,-0.145540,2.059483,0.960419,-0.119086,-0.110977,0.033922,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
249996,9.074680e-01,3.424208,-1.277992,-4.147286,-0.145540,2.059483,0.960419,-0.119086,-0.110977,0.033922,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
249997,9.373193e-01,3.424208,-1.277992,-4.147286,-0.145540,2.059483,0.960419,-0.119086,-0.110977,0.033922,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272
249998,9.681525e-01,3.424208,-1.277992,-4.147286,-0.145540,2.059483,0.960419,-0.119086,-0.110977,0.033922,...,-0.503919,-0.701489,0.077404,0.026496,-0.039385,-0.041521,-0.008651,0.031465,-0.024288,-0.004272


In [8]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/')
except:
    pass

In [9]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [10]:
data_id    = 'res'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput        = DataNoZero[InputVar]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput       = DataNoZero[['t'] + Vars]
DataOutput[Vars] = DataOutput[Vars].to_numpy() * 0.
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [11]:
data_id = 'ics'

Data_ics                     = Data[Data['t'] == FixedMinVal]
n_points_ics                 = len(Data_ics)

idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

n_valid                      = len(valid_idx)
n_train                      = len(train_idx)

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
except:
    pass

DataInput      = Data_ics[InputVar]
DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

DataOutput     = Data_ics[['t'] + Vars]
DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [12]:
# data_id = 'scs'

# Mask                         = [i for i in range(1,len(Data)-1) if Data.iloc[i-51]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]
# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=41)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput     = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)

In [13]:
# data_id = 'fcs'

# Mask                         = [i for i in range(-1,len(Data)-1) if Data.iloc[i+1]['t'] == FixedMinVal]
# Data_ics                     = Data.iloc[Mask]

# n_points_ics                 = len(Data_ics)
# idx_ics                      = np.arange(n_points_ics)
# train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=valid_perc/100, random_state=43)

# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/')
# except:
#     pass
# try:
#     os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/')
# except:
#     pass

# DataInput      = Data_ics[InputVar]
# DataInput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Input.csv', index=False)
# DataInput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Input.csv', index=False)

# DataOutput       = Data_ics[['t'] + Vars]
# DataOutput.iloc[train_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/train/'+data_id+'/Output.csv', index=False)
# DataOutput.iloc[valid_idx_ics].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/valid/'+data_id+'/Output.csv', index=False)